# GRU with Volume and ReLU

*This notebook enhances the GRU model by including 'Volume' as an input feature and using ReLU for hidden states. It compares how volume affects model accuracy and convergence.*

### Import Libraries, Download Data, Stock Data

In [1]:
import numpy as np
import pandas as pd
import yfinance as yf
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense
from tensorflow.keras.optimizers import Adam

/Users/elizabethlarez/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
ticker = "AAPL"
data = yf.download(ticker, start="2010-01-01", end="2023-11-13")
df = data[['Close', 'Volume']].reset_index()

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed


### Preprocess Data and Create Sequences

In [3]:
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(df[['Close', 'Volume']])

def create_sequences(data, seq_length):
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data[i:(i + seq_length), :])
        y.append(data[i + seq_length, 0])
    return np.array(X), np.array(y)

seq_length = 60
X, y = create_sequences(scaled_data, seq_length)

### Split and Reshape Data

In [4]:
train_size = int(len(X) * 0.8)
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

### Build and Train Model

In [5]:
model = Sequential([
    GRU(50, return_sequences=True, input_shape=(seq_length, 2), activation='relu', recurrent_activation='sigmoid'),
    GRU(50, return_sequences=False, activation='relu', recurrent_activation='sigmoid'),
    Dense(25, activation='relu'),
    Dense(1)
])
model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')
model.fit(X_train, y_train, batch_size=32, epochs=100, validation_split=0.1, verbose=1)

Epoch 1/100


/Users/elizabethlarez/Library/Python/3.9/lib/python/site-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


78/78 ━━━━━━━━━━━━━━━━━━━━ 3s 20ms/step - loss: 0.0048 - val_loss: 0.0113
Epoch 2/100
78/78 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - loss: 1.2360e-04 - val_loss: 0.0053
Epoch 3/100
78/78 ━━━━━━━━━━━━━━━━━━━━ 2s 19ms/step - loss: 5.2005e-05 - val_loss: 0.0052
Epoch 4/100
78/78 ━━━━━━━━━━━━━━━━━━━━ 2s 21ms/step - loss: 2.4509e-05 - val_loss: 0.0054
Epoch 5/100
78/78 ━━━━━━━━━━━━━━━━━━━━ 2s 20ms/step - loss: 1.8536e-05 - val_loss: 0.0052
Epoch 6/100
78/78 ━━━━━━━━━━━━━━━━━━━━ 2s 20ms/step - loss: 2.1257e-05 - val_loss: 0.0052
Epoch 7/100
78/78 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - loss: 1.7722e-05 - val_loss: 0.0055
Epoch 8/100
78/78 ━━━━━━━━━━━━━━━━━━━━ 2s 19ms/step - loss: 2.0365e-05 - val_loss: 0.0053
Epoch 9/100
78/78 ━━━━━━━━━━━━━━━━━━━━ 2s 19ms/step - loss: 2.0392e-05 - val_loss: 0.0051
Epoch 10/100
78/78 ━━━━━━━━━━━━━━━━━━━━ 2s 19ms/step - loss: 1.9421e-05 - val_loss: 0.0061
Epoch 11/100
78/78 ━━━━━━━━━━━━━━━━━━━━ 2s 19ms/step - loss: 2.9072e-05 - val_loss: 0.0056
Epoch 12/100
78/78 ━━━━━

### Make Predictions and RMSE

In [6]:
predictions = model.predict(X_test)
predictions = scaler.inverse_transform(np.concatenate((predictions, np.zeros_like(predictions)), axis=1))[:, 0]
y_test = scaler.inverse_transform(np.concatenate((y_test.reshape(-1, 1), np.zeros_like(y_test.reshape(-1, 1))), axis=1))[:, 0]
rmse = np.sqrt(np.mean((predictions - y_test)**2))
print(f"GRU + Volume + ReLU RMSE: {rmse}")

22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
GRU + Volume + ReLU RMSE: 22.141479473567955
